#Load dataset

In [5]:
%pwd

'e:\\SpringBoot\\DoAnNganh\\Medical_News\\coreRAG_chatbot\\research'

In [6]:
import os
os.chdir('../')


In [3]:
%pwd   # -- check OK :   'e:\\SpringBoot\\DoAnNganh\\Medical_News\\coreRAG_chatbot'


'e:\\SpringBoot\\DoAnNganh\\Medical_News\\coreRAG_chatbot'

In [3]:

from langchain.document_loaders import DirectoryLoader , UnstructuredWordDocumentLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter    


def load_word_files(data):
    loader = DirectoryLoader(
        path=data,
        glob="*.docx",
        loader_cls=UnstructuredWordDocumentLoader #loại file cần load   
    )
    documents = loader.load()
    return documents


In [4]:
extracted_data = load_word_files(data='DataChatbot/')

In [5]:
extracted_data

[Document(metadata={'source': 'DataChatbot\\10 căn bệnh nguy hiểm nhất trong xã hội hiện nay _ Vinmec.docx'}, page_content='Embedded Content\n\n[Trang chủ](https://www.vinmec.com/vie/) [Chuyên trang sức khoẻ](https://www.vinmec.com/vie/chuyen-trang-suc-khoe/) [Sức khoẻ tổng quát](https://www.vinmec.com/vie/suc-khoe-tong-quat/)\n\n# 10 căn bệnh nguy hiểm nhất trong xã hội hiện nay\n\n☰Mục lục\n\n1\\. Bệnh thiếu máu cơ tim, hoặc bệnh mạch vành\n\n2\\. Đột quỵ\n\n3\\. Nhiễm trùng đường hô hấp dưới\n\n4\\. Bệnh phổi tắc nghẽn mãn tính\n\n5\\. Ung thư khí quản, phế quản và phổi\n\n6\\. Đái tháo đường\n\n7\\. Bệnh Alzheimer và các chứng sa sút trí tuệ khác\n\n8\\. Mất nước do bệnh tiêu chảy\n\n9\\. Bệnh lao\n\n10\\. Xơ gan\n\n**Bài viết bởi Thạc sĩ, Bác sĩ Mai Viễn Phương - Bác sĩ nội soi tiêu hóa - Khoa Khám bệnh & Nội khoa - Bệnh viện Đa khoa Quốc tế Vinmec Central Park**\n\n**Khi mọi người nghĩ về những căn bệnh nguy hiểm nhất trên thế giới, họ sẽ nghĩ ngay tới những căn bệnh nan y, có tá

#Preprocessing (Tiền xử lý dữ liệu)


In [6]:
import re

def preprocess_data(text):

    text = re.sub(r'http\S+', '', text)
    #Xoá các ký tự bảng markdown (|, ---)
    text = re.sub(r'\|.*?\|', '', text)
    text = re.sub(r'-{2,}', '', text)
    #Xóa emoji và Unicode không cần thiết
    text = re.sub(r'[^\w\s,.!?à-ỹÀ-Ỹ\-–]', '', text)

    text = re.sub(r'\s+', ' ', text).strip()
    return text
    
    

In [7]:
from langchain.schema import Document

cleaned_data = []
for doc in extracted_data:
    cleaned_content = preprocess_data(doc.page_content)
    cleaned_doc = Document(
        page_content=cleaned_content,
        metadata=doc.metadata 
    )
    cleaned_data.append(cleaned_doc)




In [8]:
cleaned_data

[Document(metadata={'source': 'DataChatbot\\10 căn bệnh nguy hiểm nhất trong xã hội hiện nay _ Vinmec.docx'}, page_content='Embedded Content Trang chủ Chuyên trang sức khoẻ Sức khoẻ tổng quát 10 căn bệnh nguy hiểm nhất trong xã hội hiện nay Mục lục 1. Bệnh thiếu máu cơ tim, hoặc bệnh mạch vành 2. Đột quỵ 3. Nhiễm trùng đường hô hấp dưới 4. Bệnh phổi tắc nghẽn mãn tính 5. Ung thư khí quản, phế quản và phổi 6. Đái tháo đường 7. Bệnh Alzheimer và các chứng sa sút trí tuệ khác 8. Mất nước do bệnh tiêu chảy 9. Bệnh lao 10. Xơ gan Bài viết bởi Thạc sĩ, Bác sĩ Mai Viễn Phương - Bác sĩ nội soi tiêu hóa - Khoa Khám bệnh Nội khoa - Bệnh viện Đa khoa Quốc tế Vinmec Central Park Khi mọi người nghĩ về những căn bệnh nguy hiểm nhất trên thế giới, họ sẽ nghĩ ngay tới những căn bệnh nan y, có tác dụng nhanh chóng. Nhưng trên thực tế, nhiều loại bệnh này không nằm trong top 10 nguyên nhân gây tử vong trên toàn thế giới. Ước tính 56,4 triệu người đã qua đời trên toàn thế giới vào năm 2015, và 68 trong s

#Tách thành các chunk


In [9]:
def text_split(cleaned_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(cleaned_data)
    return text_chunks

In [10]:
text_chunks = text_split(cleaned_data)
print(f'So luong chunk da tach', len(text_chunks))

So luong chunk da tach 119


Embeddings vector -> (Vietnamese)  768-dimensional vector space -> author nói 768 chiều

In [11]:
from langchain.embeddings import HuggingFaceEmbeddings


def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="dangvantuan/vietnamese-embedding"
    )
    return embeddings




In [13]:
embeddings = download_huggingface_embeddings()

C:\Users\thean\AppData\Local\Temp\ipykernel_7384\3523524361.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [14]:
query_result = embeddings.embed_query("Hà Nội là thủ đô của Việt Nam")
print("Chiều của vector", len(query_result))

Chiều của vector 768


#IMPORT KEY 

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
DEEPSEEK_API_KEY=os.environ.get('DEEPSEEK_API_KEY')



#Tạo db trên pinecone -> lưa vector embeddings

-Tạo bên mail cá nhân

In [ ]:
#Tạo 
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec
import os


pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"

pc.create_index(
    name=index_name,  #ten db
    dimension=768,   #Số chiều vector theo docs 
    metric="cosine",   #So sánh theo vector theo cosin 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "medical-chatbot",
    "metric": "cosine",
    "host": "medical-chatbot-vv3hd7x.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 768,
    "deletion_protection": "disabled",
    "tags": null
}

In [9]:
import os 
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY #gán dùng lâu dài
os.environ["DEEPSEEK_API_KEY"] = DEEPSEEK_API_KEY

index_name = "medical-chatbot"

In [32]:
#Đưa chunk về vector bằng model đã tải và lưu và pinecone 

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks, # data đã clean và tách chunk
    index_name=index_name, 
    embedding=embeddings, #mô hình dangvantuan/vietnamese-embedding
)



In [15]:
#Load các vector db đã lưu  
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings, #mô hình dangvantuan/vietnamese-embedding
)

In [19]:
docsearch

Tìm kiếm các kq 

In [16]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})  #search_type="similarity tìm kiếm theo cosin  ,  
#as_retrieve biến docsearch thành bộ tìm kiếm , .. k =3 -> tìm 3 giá trị gần nhất

In [17]:
#Test 

retriever_docs = retriever.invoke("Triệu chứng của bệnh lao?")  #invoke() = tạo embedding + tìm kiếm trong Pinecone + trả về các tài liệu giống nhất.

In [18]:
retriever_docs

[Document(id='4b987c3a-342b-45c8-8893-b49e76077992', metadata={'source': 'DataChatbot\\10 căn bệnh nguy hiểm nhất trong xã hội hiện nay _ Vinmec.docx'}, page_content='Embedded Content Trang chủ Chuyên trang sức khoẻ Sức khoẻ tổng quát 10 căn bệnh nguy hiểm nhất trong xã hội hiện nay Mục lục 1. Bệnh thiếu máu cơ tim, hoặc bệnh mạch vành 2. Đột quỵ 3. Nhiễm trùng đường hô hấp dưới 4. Bệnh phổi tắc nghẽn mãn tính 5. Ung thư khí quản, phế quản và phổi 6. Đái tháo đường 7. Bệnh Alzheimer và các chứng sa sút trí tuệ khác 8. Mất nước do bệnh tiêu chảy 9. Bệnh lao 10. Xơ gan Bài viết bởi Thạc sĩ, Bác sĩ Mai Viễn Phương - Bác sĩ nội soi tiêu hóa - Khoa Khám bệnh Nội'),
 Document(id='7f396af2-ab7f-4cfd-992d-1e8276c45c6e', metadata={'source': 'DataChatbot\\Danh sách 47 bệnh truyền nhiễm thường gặp và độ nguy hiểm.docx'}, page_content='thiết yếu, nâng cao sức đề kháng cho cơ thể. 37. Bệnh dịch hạch Bệnh dịch hạch gây ra bởi vi khuẩn Yersinia pestis. Đây là một trong các bệnh truyền nhiễm tối nguy 

In [19]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model="deepseek/deepseek-r1-distill-llama-70b:free",
    openai_api_key=DEEPSEEK_API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.4,
    max_tokens=500
) #độ sáng tạo là 0.4 và số kí tự tối đa là 500


C:\Users\thean\AppData\Local\Temp\ipykernel_7384\751278041.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [21]:
#Prompt cho LLM (RAG)



from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


#Tạo mẫu nhắc nhở cho LLM
system_prompt = (
    "Bạn là một trợ lý để trả lời các câu hỏi. "
    "Hãy sử dụng các đoạn ngữ cảnh được truy xuất sau đây để trả lời "
    "câu hỏi. Nếu bạn không biết câu trả lời, hãy nói rằng bạn "
    "không biết. Trả lời tối đa ba câu và giữ cho câu trả lời "
    "ngắn gọn."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")  # dữ liệu vào
    ]
)


In [22]:
question_awnser_chain = create_stuff_documents_chain(llm,prompt) 
rag_chain = create_retrieval_chain(retriever,question_awnser_chain) # kết quả model trả ra , llm , và prompt

In [ ]:
response = rag_chain.invoke( {"input" :"Triệu chứng của bệnh lao?"}) 
print(response['answer'])

{'input': 'Triệu chứng của bệnh lao?', 'context': [Document(id='4b987c3a-342b-45c8-8893-b49e76077992', metadata={'source': 'DataChatbot\\10 căn bệnh nguy hiểm nhất trong xã hội hiện nay _ Vinmec.docx'}, page_content='Embedded Content Trang chủ Chuyên trang sức khoẻ Sức khoẻ tổng quát 10 căn bệnh nguy hiểm nhất trong xã hội hiện nay Mục lục 1. Bệnh thiếu máu cơ tim, hoặc bệnh mạch vành 2. Đột quỵ 3. Nhiễm trùng đường hô hấp dưới 4. Bệnh phổi tắc nghẽn mãn tính 5. Ung thư khí quản, phế quản và phổi 6. Đái tháo đường 7. Bệnh Alzheimer và các chứng sa sút trí tuệ khác 8. Mất nước do bệnh tiêu chảy 9. Bệnh lao 10. Xơ gan Bài viết bởi Thạc sĩ, Bác sĩ Mai Viễn Phương - Bác sĩ nội soi tiêu hóa - Khoa Khám bệnh Nội'), Document(id='7f396af2-ab7f-4cfd-992d-1e8276c45c6e', metadata={'source': 'DataChatbot\\Danh sách 47 bệnh truyền nhiễm thường gặp và độ nguy hiểm.docx'}, page_content='thiết yếu, nâng cao sức đề kháng cho cơ thể. 37. Bệnh dịch hạch Bệnh dịch hạch gây ra bởi vi khuẩn Yersinia pestis.